In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

Requirement already up-to-date: tables in /usr/local/lib/python3.6/dist-packages (3.6.1)


In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [23]:
cd  "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two


In [0]:
df = pd.read_hdf('data/car.h5')

##Feature Enginnering


In [0]:
SUFFIX_CAT = '__cat'

for feat in df.columns:
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [26]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [27]:
X = df[cat_feats].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
score = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

np.mean(score), np.std(score)

(-19695.13091100928, 148.72570644015792)

In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

## DecisionTree

In [29]:
run_model( DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

##Random Forest

In [30]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model( model, cat_feats)

(-18718.657185256638, 64.5424578125788)

##XGBoost

In [31]:
xgb_params = {
    'max_depth': 5, 
    'n_estimators': 50, 
    'learning_rate': 0.1,
    'seed': 0
}

model = xgb.XGBRegressor(**xgb_params)
run_model( model, cat_feats)

[23:26:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:26:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:26:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [32]:
m_xgb_params = {
    'max_depth': 5, 
    'n_estimators': 50, 
    'learning_rate': 0.1,
    'seed': 0
}

m = xgb.XGBRegressor(**m_xgb_params)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[23:27:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [34]:
feats = [
         'param_napęd__cat',
         'param_rok-produkcji__cat',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc__cat',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat',
]

model = xgb.XGBRegressor(**xgb_params)
run_model( model, feats)

[23:35:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:35:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:35:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13490.914910427207, 160.2569622546288)

In [33]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)

feats = [
         'param_napęd__cat',
         'param_rok-produkcji',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc__cat',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat',
]

model = xgb.XGBRegressor(**xgb_params)
run_model( model, feats)

[23:35:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:35:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:35:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11355.584151791176, 66.00730218369102)

In [36]:
# df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)

feats = [
         'param_napęd__cat',
         'param_rok-produkcji',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa__cat',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat',
]

model = xgb.XGBRegressor(**xgb_params)
run_model( model, feats)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: '218 KM'

  FitFailedWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: '90 KM'

  FitFailedWarning)


(nan, nan)

In [37]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(str(x).split('cm')[0].replace(' ', '')))
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else x)

feats = [
         'param_napęd__cat',
         'param_rok-produkcji',
         'param_stan__cat',
         'param_skrzynia-biegów__cat',
         'param_faktura-vat__cat',
         'param_moc',
         'param_marka-pojazdu__cat',
         'feature_kamera-cofania__cat',
         'param_typ__cat',
         'param_pojemność-skokowa',
         'feature_wspomaganie-kierownicy__cat',
         'param_model-pojazdu__cat',
         'param_wersja__cat',
         'param_kod-silnika__cat',
         'feature_system-start-stop__cat',
         'feature_asystent-pasa-ruchu__cat',
         'feature_czujniki-parkowania-przednie__cat',
         'feature_łopatki-zmiany-biegów__cat',
         'feature_regulowane-zawieszenie__cat',
]

model = xgb.XGBRegressor(**xgb_params)
run_model( model, feats)

[23:36:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:36:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:36:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9620.877948694586, 92.0101826431619)